### Flask App for CO_2 solubility

In [3]:
pip install scikit-learn==1.2.2

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn==1.2.2 from https://files.pythonhosted.org/packages/db/98/169b46a84b48f92df2b5e163fce75d471f4df933f8b3d925a61133210776/scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.3 MB 2.3 MB/s eta 0:00:04
   - -------------------------------------- 0.3/8.3 MB 4.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.3 MB 9.6 MB/s eta 0:00:01
   ----------------- ---------------------- 3.7/8.3 MB 21.4 MB/s eta 0:00:01
   -------------------------------- ------- 6.7/8.3 MB 30.5 MB/s eta 0:00:01
   ------------------------------------- -- 7.7/8.3 MB 27.4 MB/s eta 0:00:01
   ---------------------------------------  8.1/8.3 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 8.3/8.3 MB 24.0 MB/s eta 0:00:00
Note: you may n

In [ ]:
### Experimenting

In [ ]:
from flask import Flask, request, jsonify, render_template
import pickle
import numpy as np
import os
import nest_asyncio  # Required for running Flask in Jupyter Notebook

# Apply nest_asyncio to avoid conflicts with Jupyter Notebook
nest_asyncio.apply()

# Define file paths for the models
rf_pure_model_path = "C:/Users/x938l253/Desktop/Final CO2 APP CODE/Optimized/yhup/pure_rf.pkl"
gb_pure_model_path = "C:/Users/x938l253/Desktop/Final CO2 APP CODE/Optimized/yhup/pure_gb.pkl"
rf_brine_model_path = "C:/Users/x938l253/Desktop/Final CO2 APP CODE/Optimized/yhup/brine_rf.pkl"
gb_brine_model_path = "C:/Users/x938l253/Desktop/Final CO2 APP CODE/Optimized/yhup/brine_gb.pkl"

# Load the trained models
with open(rf_pure_model_path, "rb") as file:
    rf_pure_model = pickle.load(file)

with open(gb_pure_model_path, "rb") as file:
    gb_pure_model = pickle.load(file)

with open(rf_brine_model_path, "rb") as file:
    rf_brine_model = pickle.load(file)

with open(gb_brine_model_path, "rb") as file:
    gb_brine_model = pickle.load(file)

# Initialize Flask app and set template folder
app = Flask(__name__, template_folder='templates')

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON data from request
        data = request.get_json()

        # Validate presence of required keys
        required_keys = ["pressure", "temperature", "model"]
        for key in required_keys:
            if key not in data or data[key] is None:
                return jsonify({"error": f"Missing or null value for '{key}'"}), 400

        # Extract input values
        P_MPa = float(data["pressure"])          # Pressure in MPa
        T_K = float(data["temperature"])         # Temperature in Kelvin
        if P_MPa <= 0 or T_K <= 0:
            return jsonify({"error": "Pressure and Temperature must be positive values."}), 400

        # Set Ionic Strength: If missing, set to 0 for Pure Models
        selected_model = data["model"]
        if selected_model in ["Pure_RF", "Pure_GB", "Pure_Ensemble"]:
            Ionic_Strength = 0  # Force Ionic Strength to 0 for Pure models
        else:
            # For Brine models, ensure Ionic Strength is provided
            if "ionic_strength" not in data or data["ionic_strength"] is None:
                return jsonify({"error": "Ionic Strength is required for Brine models."}), 400
            Ionic_Strength = float(data["ionic_strength"])
            if Ionic_Strength < 0:
                return jsonify({"error": "Ionic Strength must be a positive value."}), 400

        # Format data as a NumPy array (no transformations applied)
        input_data = np.array([[P_MPa, T_K, Ionic_Strength]])

        # Select the appropriate model
        if selected_model == "Pure_RF":
            prediction = rf_pure_model.predict(input_data)[0]
        elif selected_model == "Pure_GB":
            prediction = gb_pure_model.predict(input_data)[0]
        elif selected_model == "Brine_RF":
            prediction = rf_brine_model.predict(input_data)[0]
        elif selected_model == "Brine_GB":
            prediction = gb_brine_model.predict(input_data)[0]
        elif selected_model == "Pure_Ensemble":
            pred_rf = rf_pure_model.predict(input_data)[0]
            pred_gb = gb_pure_model.predict(input_data)[0]
            prediction = (pred_rf + pred_gb) / 2  # Averaging predictions
        elif selected_model == "Brine_Ensemble":
            pred_rf = rf_brine_model.predict(input_data)[0]
            pred_gb = gb_brine_model.predict(input_data)[0]
            prediction = (pred_rf + pred_gb) / 2  # Averaging predictions
        else:
            return jsonify({"error": "Invalid model selected"}), 400

        # Return prediction as JSON with 5 decimal places
        return jsonify({"prediction": round(prediction, 5)})

    except ValueError as ve:
        return jsonify({"error": f"Invalid input value: {ve}"}), 400
    except Exception as e:
        return jsonify({"error": f"An error occurred: {e}"}), 500
    
# Run Flask in Jupyter Notebook
from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Mar/2025 10:34:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2025 10:34:47] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\x938l253\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
127.0.0.1 - - [12/Mar/2025 10:34:57] "POST /predict HTTP/1.1" 200 -
